<a href="https://colab.research.google.com/github/abelowska/dataPy/blob/main/Classes_08_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model selection: in a search for the best model

In this notebook you will learn various techniques to help you in the model selection procedure:
- [Cross-validation](https://machinelearningmastery.com/k-fold-cross-validation/) procedure used to evaluate machine learning models on a limited data sample.
- [Grid search](https://medium.com/fintechexplained/what-is-grid-search-c01fe886ef0a) used to find the optimal hyperparameters of a model which results in the most ‘accurate’ predictions.
- Statistical tests for comparing models' performance.

---
**Let's find the best model of *Orthodoxy ~ Extraversion+ Agreeableness + Conscientiousness + Openness + Neuroticism***

Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid", palette="deep")

import io
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import PredictionErrorDisplay, median_absolute_error

from scipy import stats

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# parameters for plotting
plt.rcParams["figure.figsize"] = (10,7)

In [ ]:
# constants
test_size=0.2
random_state=42

In [ ]:
def compute_score(y_true, y_pred):
  '''
  Helper function for printing scores.

  Parameters:
  y_true: ndarray of y values from original dataset.
  y_pred: ndarray of y values predicted with given model.

  Return:
  dictionary object that consists of R2 and median absolute error scores.

  '''
  return {
        "R2": r2_score(y_true, y_pred),
        "MedianAE": median_absolute_error(y_true, y_pred),
}

## Load dataset

In [ ]:
df = pd.read_csv('data_neo-ffi_religion.csv')
df['Orthodoxy'] = np.log(df[['Orthodoxy']].to_numpy())
df.head()

### Prepare data

Inspect the dataset

In [ ]:
df.describe()

Create classes

In [ ]:
df['class'] = df[['External Critique', 'Orthodoxy', 'Relativism', 'Second Naïveté']].idxmax(axis=1)

In [ ]:
df.head()

Select X and y sets

In [ ]:
X = df[[
    'Extraversion',
    'Agreeableness',
    'Conscientiousness',
    'Openness',
    'Neuroticism'
    ]]

y = df[['Orthodoxy']]

Train-test split

In [ ]:
# to ensure repeatability of splits, we set the random state
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=test_size,
    random_state=random_state
)

print(f"Shape of the X train dataset: {X_train.shape}")
print(f"Shape of the X test dataset: {X_test.shape}")
print(f"Shape of the y train dataset: {y_train.shape}")
print(f"Shape of the y test dataset: {y_test.shape}")

---
## Best model: Problem 1 - Choosing

It is quite obvious that we would like to find the best model of *Orthodoxy vs Big Five*. As we saw during the last classes for e.g., SVM we can set the values at least two hyperparameters - and there is a lot of values to test! What model - what hyperparameters - are the best?

**And what does it mean - *\"the best model\"*?**

As everywhere, we have to have a metric that says what is better. Until now, when we said *\"the best model\"*, we were comparing the results on a **test set**. But the test set should not be used for comparison.

What is the test set for?

We should use the test set only for the final evaluation of the best model. Otherwise, there is a leakage of knowledge from the test set - it is used to choose the best model (i.e. fit!) not for pure evaluation.

So - we have to choose the best model to and **then** perform a final testing. Let's try to do this!

### Exercise 1. Comparing models based on the training dataset

Create a list of kernels to test and C to test (and epsilons to test) and using for loops try to find the best SVM model.Use only **training set** to choose the best SVM model.

In [ ]:
# your code

In [ ]:
results_df

Which model is the best?

Let's check the validation (test) scores of models and check if we were right.

In [ ]:
results_test_df = results_df.copy()

# create an empty column for r2 score on test set
results_test_df['test_r2_score'] = 0

for idx, row in results_df.iterrows():
  # get model form dataframe
  model = row['model']

  # evaluate model on the test set
  y_test_pred = model.predict(X_test)
  r2 = compute_score(y_test, y_test_pred)['R2']

  # save results to dataframe
  results_test_df.at[idx, 'test_r2_score'] = r2

display(results_test_df)

It is clear that some models are overfitted.


**Obviously, given the phenomenon of overfitting, the model with the best score on the training set is not the model with the best score on the test set!!!**

Cross-validation provides better estimates of the real prodictive performance of tested models.

### Exercise 2. Cross-validation
See the documentation of calculating the cross-validated scores in scikit-learn: [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) and then:
1. Add estimation of the cross-validated scores to the code below.
2. Add the mean CV score to the dataframe with the results.

In [ ]:
C_list = [0.0001, 0.1, 1.0, 10000]
epsilon_list = [0.1, 1, 2]
kernel_list = ['linear', 'rbf']

results_cv_df = pd.DataFrame()

for C in C_list:
  for epsilon in epsilon_list:
    for kernel in kernel_list:
      estimator = SVR(C=C, epsilon=epsilon, kernel=kernel)

      # create pipeline
      model = make_pipeline(StandardScaler(), estimator)

      # fit model
      model.fit(X_train, y_train.to_numpy().ravel())

      # predict on the training data
      y_train_pred = model.predict(X_train)
      r2 = compute_score(y_train, y_train_pred)['R2']

      # compute cross validated r2 scores on train data
      k_folds = 3
      cv_scores = # todo
      mean_cv_score = np.mean(cv_scores)

      # define model name
      model_name = f"kernel: {kernel}, C:{C}, epsilon: {epsilon}"

      # save results in dataframe
      this_result = pd.DataFrame({
          "model_name": [model_name],
          "train_r2_score": [r2],
          'cv_r2_score': # todo
          "model": [model],
      })

      results_cv_df = pd.concat([results_cv_df, this_result], ignore_index=True)

In [ ]:
results_cv_df

Which model is the best?

Let's check the validation (test) scores of models.

In [ ]:
results_cv_test_df = results_cv_df.copy()

# create an empty column for r2 score on test set
results_cv_test_df['test_r2_score'] = 0

for idx, row in results_cv_df.iterrows():
  # get model form dataframe
  model = row['model']

  # evaluate model on the test set
  y_test_pred = model.predict(X_test)
  r2 = compute_score(y_test, y_test_pred)['R2']

  # save results to dataframe
  results_cv_test_df.at[idx, 'test_r2_score'] = r2

display(results_cv_test_df)

Take a look at $R^2$ *~ models* plot. You can see that CV scores more or less follow test scores, **especially for the overfitting case**.

In [ ]:
df_melted = pd.melt(results_cv_test_df, id_vars=['model_name'], value_vars=['train_r2_score', 'cv_r2_score', 'test_r2_score'])

sns.lineplot(df_melted, y='value', x='model_name', hue='variable')

# Customize the plot
plt.title('R2 Scores Comparison')
plt.xlabel('Model Name')
plt.ylabel('R2 Score')
plt.legend()
plt.grid(True)
plt.xticks(rotation=90)
plt.tight_layout()

Now we see that with cross-validation we better choose our best model. CV score is much more reliable than a simple test score. This is very important especially when comparing different models before final testing.
- **Relying only on the train score can lead to an overestimation of the model's performance and predictive power.**
- **Relying on the test score lead to knowledge leaking and loss of generalisability.**

### (Exercise 2.1) Compare mean CV scores for different number of folds

Models fit always depends on the dataset (as you remember from one of the homeworks). The more folds, the larger the set the model fits on, but the smaller the set for internal testing. Test whether (and how) the number of folds, and therefore the size of the internal sets for training and testing, affects the quality of the model. Compare the results to the r2 of the training and testing set.

In [ ]:
# your code

In [ ]:
# your plotting code

---
## Best model: Problem 2 - Searching and choosing

Using the SVR example, we saw that in certain estimators we can set certain parameters that change the way the data is fit. These parameters are called **hyperparameters** of the model.

It is intuitive, that there is an optimal composition of hyperparameters that yield the best score. It is quite hard to find optimal hyperparameters for an estimator, considering how huge a space we have to search.

Finding optimal hyperparameters is an optimization problem - and there are some techniques that help us to search over specified hyperparameters values for an estimator.

One of such techniques is so called [`Grid Search`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). `Grid Search` performs exhaustive search over specified parameter values for an estimator.

Let's try to use `GridSearch` to find optimal hyperparameters values for estimators.

- `GridSearch` works exactly the same as `Pipelines` - we can `fit()` and `predict()` on `GridSearch` object.
- `GridSearch` is parametrised with so called `param_grid` - the dictionary of parameters, where keys are estimator's parameters (hyperparameters) and values are lists of huperparameters' values to test, e.g.:

```
svr_params = dict(
    svr__kernel=["rbf"],
    svr__C=[1, 10],
    svr__epsilon=[2, 5, 10],
)
```
- You can pass `scoring` parameter to `GridSearch`, to set the metric of performance evaluation, e.g., `scoring = "r2"`.
- You can pass the strategy of CV splitting, or just the number of folds, e.g., `cv=5`.



### Exercise 3: Serch for SVR best hyperparameters

In [ ]:
svr_params = # define a grid of parameters to be searched

In [ ]:
# create pipeline
model = make_pipeline(StandardScaler(), SVR())

# define cross-validation k
cv_kf = # todo

# define grid search
grid_search_model = GridSearchCV(
    # todo
)


# fit model
grid_search_model.fit(X_train, y_train)

# predict on test data
y_test_pred = grid_search_model.predict(X_test)
test_score = r2_score(y_test, y_test_pred)
print(f'Test R2 score: {test_score}')

# predict on train data
y_train_pred = grid_search_model.predict(X_train)
train_score = r2_score(y_train, y_train_pred)
print(f'Train R2 score: {train_score}')

# extract mean cv scores
mean_cv_score = grid_search_model.best_score_
print(f'CV mean R2 score: {mean_cv_score}')

View results of Grid Search:

In [ ]:
print(f"Choosen model: {grid_search_model.best_estimator_}\n")
print(f"Choosen hyperparameters: {grid_search_model.best_params_}\n")
print(f"Train score: {train_score} \nTest score: {test_score}")

Look at the Grid Search results in details:

In [ ]:
cv_results_df = pd.DataFrame(grid_search_model.cv_results_)
cv_results_df

---
### (Exercise 3.1) Potting GS results
Try to plot results of the grid search to see how the values of the given hyperparameters affected the performance of the model. Plot e.g., $R^2$ *~ epsilon*, with hue on C values.

In [ ]:
# your code

---

In [ ]:
cv_results_df.iloc[[grid_search_model.best_index_]]

In [ ]:
cv_splits_scores = cv_results_df.filter(
    regex=r"split\d*_test_r2").iloc[grid_search_model.best_index_]

In [ ]:
print(f"Best estimator: {grid_search_model.best_estimator_}\n")
print(f"Best parameters: {grid_search_model.best_params_}\n")
print(f"Best mean CV score: {grid_search_model.best_score_}\n")
print(f"CV scores:\n{cv_splits_scores}")

**Now we know the values of hyperparameters of the model with the best mean CV score. And - most importantly - we have chosen our model without knowledge leakage for the testing set.**